In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import DBSCAN
from sklearn.metrics import adjusted_rand_score
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

# Загрузка данных
df = pd.read_csv("../data/prepared_data_for_clusterization.csv")

# Признаки
feature_columns = [
    'duration', 'frame rate', 'pixel_count',
    'title_len', 'uppercase_ratio', 'emoji_count', 'has_exclamation',
#    'category_Howto & Style', 'category_Music', 'category_News & Politics',
#    'category_Nonprofits & Activis', 'category_People & Blogs',
#    'category_Pets & Animals', 'category_Science & Technology',
#    'category_Shows', 'category_Sports', 'category_Travel & Events',
    'description', 'hashtags'
]

X = df[feature_columns].values

# Числовые и категориальные признаки
numeric = ['duration', 'pixel_count', 'title_len', 'uppercase_ratio', 'emoji_count', 'has_exclamation']
cat_cols = [c for c in df.columns if c.startswith('category_')]

# ───────────────────────────────────────────────────────
# 1. ПОДБОР ПАРАМЕТРОВ DBSCAN И ОБУЧЕНИЕ
# ───────────────────────────────────────────────────────


min_samples = 18

# Подбор eps через k-расстояния (k = min_samples)
nbrs = NearestNeighbors(n_neighbors=min_samples).fit(X)
distances, _ = nbrs.kneighbors(X)
k_distances = np.sort(distances[:, -1])[::-1]
eps = np.percentile(k_distances, 94)  # 90-й перцентиль — хороший компромисс

print(f"Подобраны параметры: eps = {eps:.3f}, min_samples = {min_samples}")

# Обучение DBSCAN
dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='euclidean', n_jobs=-1)
cluster_labels = dbscan.fit_predict(X)
df['cluster'] = cluster_labels

n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
n_noise = list(cluster_labels).count(-1)

print(f"\nОбучение завершено:")
print(f"  → Найдено кластеров: {n_clusters}")
print(f"  → Точек-выбросов (шум): {n_noise} ({n_noise / len(df) * 100:.1f}%)")

# ───────────────────────────────────────────────────────
# 2. a. ЭКСПЕРТНАЯ ОЦЕНКА
# ───────────────────────────────────────────────────────
print("\n" + "="*50)
print("a. ЭКСПЕРТНАЯ ОЦЕНКА")
print("="*50)

# Распределение по кластерам
print("\nРаспределение объектов по кластерам:")
labels_series = pd.Series(cluster_labels)
cluster_counts = labels_series.value_counts().sort_index()
for label in sorted(cluster_counts.index):
    cnt = cluster_counts[label]
    if label == -1:
        print(f"  Шум (label = -1): {cnt} видео ({cnt / len(df) * 100:.1f}%)")
    else:
        print(f"  Кластер {label}: {cnt} видео ({cnt / len(df) * 100:.1f}%)")

# Средние значения (только по реальным кластерам, шум исключён)
print("\nСредние значения числовых признаков по кластерам (шум исключён):")
non_noise_mask = cluster_labels != -1
if non_noise_mask.sum() > 0:
    cluster_means = df[non_noise_mask].groupby('cluster')[numeric].mean().round(3)
    print(cluster_means.to_string())
else:
    print("  Все точки — шум. Средние не определены.")

# Доминирующие категории в кластерах
print("\nДоля категорий в кластерах (топ-3 на кластер, шум исключён):")
if non_noise_mask.sum() > 0:
    cat_means = df[non_noise_mask].groupby('cluster')[cat_cols].mean()
    for k in sorted(cat_means.index):
        print(f"\nКластер {k}:")
        top_cats = cat_means.loc[k].nlargest(3)
        for cat, prob in top_cats.items():
            cat_name = cat.replace('category_', '')
            print(f"  • {cat_name:<25} → {prob:.3f}")
else:
    print("  Нет кластеров → категории не оцениваются.")

# ───────────────────────────────────────────────────────
# 3. b. СРАВНЕНИЕ С РЕАЛЬНЫМИ РАЗБИЕНИЯМИ
# ───────────────────────────────────────────────────────
print("\n" + "="*50)
print("b. СРАВНЕНИЕ С РЕАЛЬНЫМИ РАЗБИЕНИЯМИ")
print("="*50)

# Восстановление истинной категории
df['true_category'] = df[cat_cols].idxmax(axis=1).str.replace('category_', '')

# ARI: шум → отдельный класс ("OUTLIER")
y_true = df['true_category']
y_pred_for_ari = np.where(cluster_labels == -1, "OUTLIER", cluster_labels.astype(str))
ari_cat = adjusted_rand_score(y_true, y_pred_for_ari)
print(f"ARI (кластеры + шум vs категории): {ari_cat:.4f}")

# Сравнение с популярностью (если есть 'views')
if 'views' in df.columns:
    y_pop = (df['views'] > 200).astype(int)
    # шум → новый класс (например, 2, если y_pop ∈ {0,1})
    y_pred_pop = np.where(cluster_labels == -1, max(y_pop) + 1, cluster_labels)
    ari_pop = adjusted_rand_score(y_pop, y_pred_pop)
    print(f"ARI (кластеры vs популярность, views>200): {ari_pop:.4f}")
else:
    print("Столбец 'views' отсутствует — сравнение с популярностью невозможно.")



# ───────────────────────────────────────────────────────
# 4. c. ВИЗУАЛИЗАЦИЯ
# ───────────────────────────────────────────────────────
print("\n" + "="*50)
print("c. ВИЗУАЛИЗАЦИЯ")
print("="*50)
print("Генерация диаграмм...")

# PCA для 2D-проекции
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X)

plt.figure(figsize=(14, 5))

# 1. Топ-3 категории по кластерам (без шума)
plt.subplot(1, 2, 1)
if non_noise_mask.sum() > 0:
    clusters = sorted([k for k in cluster_counts.index if k != -1])
    x = np.arange(len(clusters))
    width = 0.8 / 3
    colors = plt.cm.tab10.colors
    for i in range(3):
        values = []
        for k in clusters:
            top_cats = cat_means.loc[k].nlargest(3)
            values.append(top_cats.iloc[i] if i < len(top_cats) else 0)
        label = f"Топ-{i+1}"
        if i == 0 and len(clusters) > 0:
            first_cat = cat_means.loc[clusters[0]].nlargest(1).index[0].replace('category_', '')
            label += f" (напр., '{first_cat}')"
        plt.bar(x + i * width - 0.4 + width/2, values, width,
                label=label, color=colors[i % len(colors)], alpha=0.8)
    plt.xlabel('Кластер')
    plt.ylabel('Доля видео')
    plt.title('Топ-3 категории в кластерах (DBSCAN)')
    plt.xticks(x, [f'Кл.{k}' for k in clusters])
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
else:
    plt.text(0.5, 0.5, 'Нет кластеров', ha='center', va='center', fontsize=14)
    plt.title('Топ-3 категории в кластерах (DBSCAN)')
    plt.axis('off')

# 2. 2D-проекция с цветами
plt.subplot(1, 2, 2)
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1],
                      c=cluster_labels, cmap='tab20', s=25, alpha=0.7,
                      edgecolors='k', linewidth=0.3)
plt.colorbar(scatter, label='Кластер (−1 = шум)')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} дисп.)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} дисп.)')
plt.title(f'DBSCAN: {n_clusters} кластеров, {n_noise} выбросов')

plt.tight_layout()
plt.savefig("dbscan_result.png", dpi=300, bbox_inches='tight')

plt.show()
